In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str,
    "sr_proceso": str,   
    "sr_subproceso": str,
    "sr_instancia_proceso": str,
    "sr_etapa": str,
    "sr_subetapa": str,
	"sr_id_snapshot": str,
	"sr_usuario": str
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "PATRIF_AIN_0100_DES_IND_DSM_FOV.sql",
    sr_folio = params.sr_folio,
    sr_subproceso = params.sr_subproceso
)

db.write_delta(f"DELTA_TRA_DES_IND_DSM_FOV_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_DES_IND_DSM_FOV_{params.sr_folio}"))

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_DATAUX.TTAFOGRAL_ETL_IND_CTA_INDV_AUX"

db.write_data(db.read_delta(f"DELTA_TRA_DES_IND_DSM_FOV_{params.sr_folio}"), table_name, "default", "append")


In [0]:
#Se ejecuta el merge


statement = query.get_statement(
    "PATRIF_AIN_0100_DES_IND_DSM_FOV_MERGE.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:

statement = query.get_statement(
    "PATRIF_AIN_0100_DES_IND_DSM_FOV_DELETE.sql",
    sr_folio=params.sr_folio
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#Se elimina la Delta para liberar espacio
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_DES_IND_DSM_FOV_{params.sr_folio}")